In [1]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017) 
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
(train_features, train_labels), (test_features, y_labels) = cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(y_labels, num_classes)

In [4]:
!wget https://github.com/qubvel/classification_models/archive/master.zip

--2019-07-17 18:02:44--  https://github.com/qubvel/classification_models/archive/master.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/qubvel/classification_models/zip/master [following]
--2019-07-17 18:02:44--  https://codeload.github.com/qubvel/classification_models/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]  41.40K  --.-KB/s    in 0.09s   

2019-07-17 18:02:44 (476 KB/s) - ‘master.zip’ saved [42397]



In [9]:
!unzip master.zip

Archive:  master.zip
e27da2b14e390b38dd6fe48f737abc07746bf3af
replace classification_models-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: classification_models-master/.gitignore  
  inflating: classification_models-master/.gitmodules  
  inflating: classification_models-master/.travis.yml  
  inflating: classification_models-master/LICENSE  
 extracting: classification_models-master/MANIFEST.in  
  inflating: classification_models-master/README.md  
 extracting: classification_models-master/__init__.py  
  inflating: classification_models-master/classification_models/__init__.py  
  inflating: classification_models-master/classification_models/__version__.py  
 extracting: classification_models-master/classification_models/common/__init__.py  
  inflating: classification_models-master/classification_models/common/blocks.py  
  inflating: classification_models-master/classification_models/keras_applications/__init__.py  
 extracting: classification_models-maste

In [0]:
!cp -r classification_models-master/* ./

In [7]:
!ls

classification_models	      LICENSE	   README.md	     setup.py
classification_models-master  MANIFEST.in  requirements.txt  tests
__init__.py		      master.zip   sample_data


In [10]:
import keras
from classification_models.resnet import ResNet18, preprocess_input

n_classes = 10

# build model
base_model = ResNet18(input_shape=(32,32,3), weights='imagenet', include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])

# train
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])


W0717 18:04:03.569062 139938679920512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 18:04:03.628658 139938679920512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 18:04:03.630218 139938679920512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0717 18:04:03.638191 139938679920512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0717 18:04:03.639088 139938

44924928/44920640 [==============================] - 2s 0us/step


W0717 18:04:11.751509 139938679920512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 32, 32, 3)    9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 16, 16, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn0 (Batch

In [12]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)


# Train the model
model_info = model.fit_generator(datagen.flow(train_features, train_labels, batch_size = 128),
                                 samples_per_epoch = train_features.shape[0], nb_epoch = 5, 
                                 validation_data = (test_features, test_labels), verbose=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., verbose=1, steps_per_epoch=390, epochs=5)`
  # Remove the CWD from sys.path while we load stuff.
W0717 18:04:12.385270 139938679920512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (f

Epoch 1/5
390/390 [==============================] - 23s 60ms/step - loss: 1.1871 - acc: 0.5927 - val_loss: 0.8914 - val_acc: 0.6896
Epoch 2/5
390/390 [==============================] - 17s 43ms/step - loss: 0.7307 - acc: 0.7456 - val_loss: 0.7533 - val_acc: 0.7417
Epoch 3/5
390/390 [==============================] - 16s 42ms/step - loss: 0.5726 - acc: 0.8005 - val_loss: 0.7236 - val_acc: 0.7590
Epoch 4/5
390/390 [==============================] - 16s 42ms/step - loss: 0.4552 - acc: 0.8433 - val_loss: 0.7085 - val_acc: 0.7667
Epoch 5/5
390/390 [==============================] - 16s 42ms/step - loss: 0.3569 - acc: 0.8783 - val_loss: 0.7187 - val_acc: 0.7692


In [0]:
score = model.evaluate(test_features, test_labels, verbose=0)

In [14]:
print(score)

[0.7186853147983551, 0.7693]


In [0]:
y_pred = model.predict(test_features)

In [16]:
print(y_pred)

[[1.19679928e-04 1.89978685e-02 4.12161433e-04 ... 4.95559012e-04
  5.89071726e-03 6.69408008e-04]
 [6.50942652e-03 1.96256712e-01 1.05688734e-04 ... 5.41531539e-04
  7.90488899e-01 4.49394621e-03]
 [1.90050453e-02 2.73479167e-02 1.99024522e-04 ... 6.67612549e-05
  9.52434540e-01 1.83511089e-04]
 ...
 [9.31475188e-06 1.01740318e-06 3.10640549e-04 ... 2.63839448e-03
  1.44553533e-05 7.70210681e-06]
 [9.46842730e-02 8.32534969e-01 3.10776439e-02 ... 2.05259246e-04
  4.36342292e-04 9.62539052e-05]
 [6.10021416e-05 1.47046201e-04 3.56422318e-03 ... 8.76504540e-01
  5.39765570e-05 1.00487414e-04]]


In [0]:
pred =[]
for i in range(len(test_features)):
  pred.append(np.argmax(y_pred[i]))


In [18]:
t_labels = []
for i in range(len(test_features)):
  t_labels.append(np.argmax(test_labels[i]))

pred = np.array(pred)

wrong_pred_id = np.where(pred != t_labels)
wrong_pred = y_labels[pred != t_labels]

print(wrong_pred_id)
print(wrong_pred)



(array([   0,    3,   12, ..., 9984, 9989, 9995]),)
[[3]
 [0]
 [5]
 ...
 [7]
 [2]
 [8]]


In [30]:
result

(array([6585]), array([1]))

In [0]:
    preds = model.predict(test_features)
    class_idx = np.argmax(preds[0])
    class_output = model.output[:, class_idx]
    last_conv_layer = model.get_layer("stage4_unit2_relu2")

In [25]:
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(512):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

InvalidArgumentError: ignored

In [26]:
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

NameError: ignored